# Exercicios

O uso do `functools.wraps` é uma boa prática ao criar decoradores em Python. Ele ajuda a preservar os metadados da função decorada, como o nome da função, docstring e assinatura, o que torna o código mais claro e mais fácil de entender. Sem usar `functools.wraps`, a função decorada pode perder essas informações importantes.

In [2]:
from functools import wraps

##### Escreva um programa Python para criar uma função decoradora para medir o tempo de execução de uma função.

In [1]:
import time

In [3]:
def stopwatch(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Comecei a calcular o tempo inicial.')
        start = time.time()
        result = func(*args, **kwargs)
        stop = time.time()
        print('Acebei de terminar de calcular o tempo final.')
        print(f"Elapsed time: {stop - start}")
        return result
    return wrapper

@stopwatch
def wait(seconds: int = 2):
    print('Entrei na função decorada')
    print(f'Vou domrir por {seconds} seconds.')
    time.sleep(seconds)
    print('Acordei')
    return None

In [4]:
# eu encubro minha função principal faço o que quero antes e depois de rodar ela, por isso usamos decorators
wait(2)

Comecei a calcular o tempo inicial.
Entrei na função decorada
Vou domrir por 2 seconds.
Acordei
Acebei de terminar de calcular o tempo final.
Elapsed time: 2.005902051925659


##### Escreva um programa Python para criar um decorador para converter o valor de retorno de uma função em um tipo de dados especificado.

In [27]:
from functools import singledispatch

# Decorator using singledispatch
def string_converter(func):
    @singledispatch
    @wraps(func)
    def wrapper(arg):
        print(f"Type: {type(arg).__name__}, Value: {arg}")
        return str(arg)

    @wrapper.register(int)
    def _(arg):
        print(f"Type: int, Value: {arg}")
        return str(arg)

    @wrapper.register(float)
    def _(arg):
        print(f"Type: float, Value: {arg}")
        return str(arg)

    @wrapper.register(list)
    def _(arg):
        print(f"Type: list, Value: {arg}")
        return ', '.join(map(str, arg))

    return wrapper

# Usage
@string_converter
def print_and_convert(value):
    return value

print_and_convert("Hello, world!")
print_and_convert(42)
print_and_convert(3.14)
print_and_convert([1, 2, 3])

Type: str, Value: Hello, world!
Type: int, Value: 42
Type: float, Value: 3.14
Type: list, Value: [1, 2, 3]


'1, 2, 3'

##### Escreva um programa Python que implemente um decorador para armazenar em cache o resultado de uma função.

In [8]:
from functools import lru_cache

@lru_cache
def sleep(seconds: int):
    time.sleep(seconds)
    return 'Finalizei'

In [9]:
start = time.time()
sleep(10)
stop = time.time()
print(f'executada em: {stop - start}')

executada em: 10.007429122924805


In [10]:
start = time.time()
sleep(10)
stop = time.time()
print(f'executada em: {stop - start}')

executada em: 0.0


#####  Escreva um programa Python que implemente um decorador para repetir uma função várias vezes.

Se o seu decorator precisa de um `parametro` crie uma função antes que receba o parametro depois crie outra com o func e por fim faça o wrapper

In [13]:
def repeat(n):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(n):
                result = func(*args, **kwargs)
                print(f"Function {func.__name__} result: {result}")
            return result
        return wrapper
    return decorator


# Decorator to repeat the function 3 times
@repeat(n=5)
def greet(name):
    """A simple greeting function."""
    return f"Hello, {name}!"

name = "Alice"
greet(name)

Hello, Alice!
Hello, Alice!
Hello, Alice!
Hello, Alice!
Hello, Alice!


'Hello, Alice!'

##### Escreva um programa Python que implemente um decorador para adicionar funcionalidade de log a uma função.

In [9]:
import logging
from functools import wraps

# Configure logging to display messages in the console
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Decorator to add logging functionality
def log_function(func):
    @wraps(func)  # Use functools.wraps to preserve original function's metadata
    def wrapper(*args, **kwargs):
        logging.info(f"Calling function {func.__name__} with arguments {args} and kwargs {kwargs}")
        result = func(*args, **kwargs)
        logging.info(f"Function {func.__name__} returned: {result}")
        return result
    return wrapper

# Using the decorator to add logging to a function
@log_function
def add(a, b):
    """Add two numbers."""
    return a + b

@log_function
def subtract(a, b):
    """Subtract two numbers."""
    return a - b

# Calling the decorated functions
result1 = add(10, 5)
result2 = subtract(10, 5)

print(add.__name__)  # Output: "add" (original function name)
print(add.__doc__)   # Output: "Add two numbers." (original docstring)

add
Add two numbers.


##### Escreva um programa Python que implemente um decorador para medir o uso de memória de uma função.

In [11]:
import tracemalloc

In [13]:
from functools import wraps

In [31]:
def memory_size(func):
    tracemalloc.start()
    @wraps(func)
    def wrapper(*args, **kwargs):
        
        initial_memory = tracemalloc.get_traced_memory()[0]
        result = func(*args, **kwargs)
        final_memory = tracemalloc.get_traced_memory()[0]
        print(f'Memory used: {final_memory - initial_memory}')
        return result
    return wrapper
        

@memory_size
def foobar(x, y):
    result = x ** y
    return result

In [34]:
foobar(2,10)

Memory used: 56


1024

# Outros exemplos

In [7]:
def foobar(func):
    @wraps(func)
    def func_123(*args, **kwargs):
        result = func(*args, **kwargs)
        print('Contagem regressiva...')
        for i in range(3,0,-1):
            print(i)
            time.sleep(1)
        return result
    return func_123

@foobar
def barfoo(name: str = 'Luiz'):
    msg = f'Olá {name}'
    return msg

barfoo()

Contagem regressiva...
3
2
1


'Olá Luiz'

#####  Contando chamadas de função com um decorador. Crie um decorador que conte quantas vezes uma função foi chamada e imprima esse número sempre que a função for executada.

In [14]:
def contar_chamadas(funcao):
    def wrapper(*args, **kwargs):
        wrapper.contagem += 1
        print(f"A função {funcao.__name__} foi chamada {wrapper.contagem} vezes.")
        return funcao(*args, **kwargs)
    wrapper.contagem = 0
    return wrapper

@contar_chamadas
def funcao_contada():
    pass

funcao_contada()
funcao_contada()
funcao_contada()

A função funcao_contada foi chamada 1 vezes.
A função funcao_contada foi chamada 2 vezes.
A função funcao_contada foi chamada 3 vezes.


55
